In [1]:
# 导入数据
import sys
sys.path.append('..')
import utils

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

多层感知机与多类逻辑回归的区别在于，多层感知机在输入层和输入层之间插入了一个到多个的隐含层

![](https://zh.gluon.ai/_images/multilayer-perceptron.png)

In [2]:
from mxnet import ndarray as nd

num_inputs = 28 * 28
num_outputs = 10

num_hidden = 256
weight_scale = 0.01

W1 = nd.random_normal(shape=(num_inputs, num_hidden), scale=weight_scale)
b1 = nd.zeros(num_hidden)

W2 = nd.random_normal(shape=(num_hidden, num_outputs), scale=weight_scale)
b2 = nd.zeros(num_outputs)

params = [W1, b1, W2, b2]

for param in params:
    param.attach_grad()

In [4]:
# 激活函数
def relu(X):
    return nd.maximum(X, 0)

In [6]:
# 定义模型
def net(X):
    X = X.reshape((-1, num_inputs))
    h1 = relu(nd.dot(X, W1) + b1)
    output = nd.dot(h1, W2) + b2
    return output

In [7]:
# softmax和损失函数
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [8]:
#训练
from mxnet import autograd

learning_rate = .5

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        utils.SGD(params, learning_rate / batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
        
    test_acc = utils.evaluate_accuracy(test_data, net)
    print('epoch %d. loss: %f, train acc %f, test acc %f' %(epoch, train_loss / len(train_data),
                                                            train_acc / len(train_data), test_acc))

epoch 0. loss: 0.805890, train acc 0.698801, test acc 0.780148
epoch 1. loss: 0.491856, train acc 0.817191, test acc 0.838542
epoch 2. loss: 0.428652, train acc 0.842849, test acc 0.855268
epoch 3. loss: 0.393585, train acc 0.854768, test acc 0.864784
epoch 4. loss: 0.372371, train acc 0.861796, test acc 0.837841
